In [80]:
import baostock as bs
import pandas as pd
import os
from pathlib import Path

OUTPUT = './'


def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


class Downloader(object):
    def __init__(self,
                 output_dir,year,quarter):
        self._bs = bs
        bs.login()
        date = '2021-04-12'
        # self.date_end = datetime.datetime.now().strftime("%Y-%m-%d")
        self.output_dir = output_dir
        self.date=date
        

    def exit(self):
        bs.logout()

    def get_codes_by_date(self, date):
        date = '2021-04-12'
        stock_rs = bs.query_all_stock(date)
        stock_df = stock_rs.get_data()
        stock_df = stock_df[stock_df.code>='sh.600000']
        stock_df = stock_df[stock_df.code<='sz.399000']
        stock_df = stock_df.reset_index()
        #stock_df = stock_df[0:2]
        print(stock_df)
        return stock_df


    def run(self):
        stock_df = self.get_codes_by_date(self.date)
        for index, row in stock_df.iterrows():
            print(f'processing {row["code"]}')
            df_code = bs.query_profit_data(row["code"], year=1990, quarter=1).get_data()
            df_code.to_csv(f'{self.output_dir}/{row["code"]}.csv', index=False)
        self.exit()
    
    def updater(self,year,quarter):
        stock_df = self.get_codes_by_date(self.date)
        for index,row in stock_df.iterrows():
            for i in year:
                for n in quarter:
                    dstdir = f'{self.output_dir}/{row["code"]}.csv'
                    if not Path(dstdir).exists():
                        df_code = bs.query_profit_data(row["code"], year=i, quarter=n).get_data()
                        if len(df_code)==0:
                            continue
                        else:
                            df_code.to_csv(f'{self.output_dir}/{row["code"]}.csv', index=False)
                        continue
                    base_code = pd.read_csv(dstdir)
                    update_code = bs.query_profit_data(row["code"], year=i,quarter=n).get_data()
                    df_code = pd.concat([base_code,update_code],axis=0,ignore_index = True)
                    df_code.to_csv(dstdir, index=False)
        self.exit()
        
if __name__ == '__main__':
    year = [1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]
    quarter = [1,2,3,4]
    down = Downloader('./', year = year,quarter=quarter)
    down.updater(year=year,quarter=quarter)



login success!
      index       code tradeStatus code_name
0       220  sh.600000           1      浦发银行
1       221  sh.600004           1      白云机场
2       222  sh.600006           1      东风汽车
3       223  sh.600007           1      中国国贸
4       224  sh.600008           1      首创股份
...     ...        ...         ...       ...
4247   4467  sz.300965           1      恒宇信通
4248   4468  sz.300966           1      共同药业
4249   4469  sz.300969           1      恒帅股份
4250   4470  sz.300970           1      华绿生物
4251   4471  sz.300999           1       金龙鱼

[4252 rows x 4 columns]
logout success!


In [78]:
bs.login()
bs.query_profit_data('sh.600025', year=2009, quarter=1).get_data()

login success!


""
